# 기본 함수

* 코드를 수정하지 말고 실행만 하세요

In [ ]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt

# 1. 특정 파일명으로 sample과 reference 이미지를 읽어오는 함수
def load_single_sample_and_reference(filename, sample_folder, reference_folder):
    sample_path = os.path.join(sample_folder, filename)
    reference_path = os.path.join(reference_folder, filename)

    sample_img = cv2.imread(sample_path)
    reference_img = cv2.imread(reference_path)

    if sample_img is None:
        raise ValueError(f"Sample image not found: {sample_path}")
    if reference_img is None:
        raise ValueError(f"Reference image not found: {reference_path}")

    return sample_img, reference_img

# 2. sample과 reference를 시각화하는 함수
def visualize_sample_and_reference(sample_img, reference_img, filename=None):
    # BGR -> RGB 변환 (OpenCV는 BGR로 읽기 때문)
    sample_img_rgb = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
    reference_img_rgb = cv2.cvtColor(reference_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(sample_img_rgb)
    plt.title('Sample Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(reference_img_rgb)
    plt.title('Reference Image')
    plt.axis('off')

    if filename:
        plt.suptitle(filename, fontsize=16)

    plt.show()

# 4. 성능 평가 (Accuracy)
def evaluate_accuracy(gt_df, pred_df):
    # --- 데이터 검증 ---
    # 같은 filename에 대해 class가 여러 개 있는지 확인
    pred_multi_class = pred_df.groupby('filename')['class'].nunique()
    if (pred_multi_class > 1).any():
        conflict_files = pred_multi_class[pred_multi_class > 1].index.tolist()
        raise ValueError(f"Prediction 데이터 오류: 다음 파일들은 여러 class를 가지고 있습니다: {conflict_files}")

    # gt_df 기준으로 left join
    merged_df = pd.merge(gt_df[['filename','class']], pred_df[['filename','class']], on='filename', how='left', suffixes=('_gt', '_pred'))

    # prediction이 없는 경우 class_pred를 'None'으로 처리
    merged_df['class_pred'] = merged_df['class_pred'].fillna('None')

    # 전체 accuracy 계산 (gt 기준)
    correct = (merged_df['class_gt'] == merged_df['class_pred']).sum()
    total = len(merged_df)
    overall_accuracy = correct / total if total > 0 else 0

    # class별 accuracy 계산
    class_acc = {}
    classes = merged_df['class_gt'].unique()

    for cls in sorted(classes):
        cls_df = merged_df[merged_df['class_gt'] == cls]
        correct_cls = (cls_df['class_gt'] == cls_df['class_pred']).sum()
        total_cls = len(cls_df)
        acc_cls = correct_cls / total_cls if total_cls > 0 else 0
        class_acc[cls] = acc_cls

    return overall_accuracy, class_acc



정답 파일 읽어오기

* gt_df 데이터 프레임에는 아래와 같은 불량에 대한 정답 정보가 들어 있습니다.
  * filename: 이미지 파일명
  * class: 불량의 종류 (6가지 중 1가지에 해당)
      * missing_hole
      * mouse_bite
      * open_circuit
      * short
      * spur
      * spurious_copper

In [ ]:
# 폴더 경로 설정
sample_folder = 'sampled_dataset/noisy_samples/'
reference_folder = 'sampled_dataset/reference/'
gt_csv_path = 'sampled_dataset/cropped_labels.csv'

# 1. GT 파일 읽기
gt_df = pd.read_csv(gt_csv_path)

## 구현

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# 1. 각 불량 유형에 대한 분류 기준 정의
def classify_defect_with_reference_comparison(mean_val, ref_mean_val, eccentricity, max_axis, min_axis, solidity, 
                                              ratio, ref_img, mask, contour, corner_count, rectangularity):
    # 1-1. 불량 밝기 차이 계산: 현재 불량 영역 밝기 - 기준 이미지 같은 영역 밝기
    brightness_diff = mean_val - ref_mean_val

    # 1-2. 밝기 차이에 따라 불량 유형 1차 분류
    if brightness_diff <= 0:  # 어두운 영역 (빠진 불량)

        # 2차 분류: 형태적 특징 및 주변 밝기 등을 바탕으로 불량 유형 구체화
        # 1-2-1. 불량이 원형에 가깝고, 형태적 비율이 중간 수준일 때 → missing_hole
        if eccentricity < 0.55 and 0.2 < ratio < 0.9:
            return 'missing_hole'

        # 1-2-2. 이전 조건에서 제외된 불량 중, 밝기 비율이 높고 사각형 유사도가 낮으면 → mouse_bite
        elif ratio > 0.4 and rectangularity < 0.8:
            return 'mouse_bite'

        # 1-2-3. 위 두 조건을 만족하지 못하면 → open_circuit
        else:
            return 'open_circuit'

    # 1-3. 밝기 차이가 양수일 경우 (밝은 영역: 추가된 불량)
    elif brightness_diff > 0:
        # 1-3-1. 불량 길이가 작고 밝기 비율이 일정 범위일 때 → short
        if max_axis < 50 and 0.3 < ratio < 1:
            return 'short'
        else:
            # 1-3-2. 주변 밝기 평균을 이용해 spur와 spurious_copper를 세부 구분.
            # spur는 주변 회로가 있어서 밝기 값이 높게 나타나고, spurious_copper는 실제 회로와 떨어진 구조물이므로 주변 밝기값이 낮게 나타남
            kernel = np.ones((3, 3), dtype=np.uint8)
            dilated = cv2.dilate(mask, kernel, iterations=3)
            border_mask = cv2.subtract(dilated, mask)
            ref_gray = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY)
            border_mean = cv2.mean(ref_gray, mask=border_mask)[0]

            # 1-3-3. 주변 밝기값이 높으면 spur, 아니면 spurious_copper
            if border_mean > 40:
                return 'spur'  
            else:
                return 'spurious_copper'  

    # 1-4. 위 모든 분류 조건에서 실패한 경우 → 분류 실패로 간주하고 None 반환
    print('분류 실패')
    return None



# 2. SIFT 매칭이 1차 시도에서 실패한 경우, 이미지 대비를 향상시켜 매칭 성능을 높이기 위한 보조 함수 정의
def enhance_image_contrast(gray_img, method='clahe'):

    if method == 'clahe':
        # CLAHE : 지역적으로 대비를 조절하여 과도한 밝기/어둠을 방지
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        return clahe.apply(gray_img)
    
    elif method == 'hist_eq':
        # 히스토그램 균등화 : 전체 이미지의 픽셀 분포를 균일하게 만들어 대비를 향상시킴
        return cv2.equalizeHist(gray_img)
    
    elif method == 'gamma':
        # 감마 보정 : 픽셀 값에 감마 함수를 적용하여 밝기를 비선형적으로 조정함.
        gamma = 0.8
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        return cv2.LUT(gray_img, table)
    
    else:
        return gray_img



# 3. SIFT 기반 이미지 정합 함수 : 입력 이미지(sample_img)를 기준 이미지(ref_img)에 정합시키기 위해 SIFT 특징점 기반으로 매칭 수행
def enhanced_sift_matching(sample_img, ref_img, filename):
    # 3-1. 두 이미지를 그레이스케일로 변환
    gray_s = cv2.cvtColor(sample_img, cv2.COLOR_BGR2GRAY)
    gray_r = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY)

    detector = cv2.SIFT_create()
    bf = cv2.BFMatcher()

    # 3-2. 원본 이미지에서 SIFT 특징점 및 기술자 추출
    kp_s, des_s = detector.detectAndCompute(gray_s, None)
    kp_r, des_r = detector.detectAndCompute(gray_r, None)

    # 3-3. 첫 번째 시도: 기본 SIFT 매칭 시도
    if des_s is not None and des_r is not None:
        try:
            matches = bf.knnMatch(des_s, des_r, k=2)
            good = [m for m, n in matches if m.distance < 0.9 * n.distance]

            # 3-4. 좋은 매칭이 3개 이상일 경우 Affine 변환 적용
            if len(good) >= 3:
                src = np.float32([kp_s[m.queryIdx].pt for m in good]).reshape(-1, 2)
                dst = np.float32([kp_r[m.trainIdx].pt for m in good]).reshape(-1, 2)
                M, _ = cv2.estimateAffine2D(src, dst, method=cv2.RANSAC, ransacReprojThreshold=4.0)

                if M is not None:
                    h_r, w_r = ref_img.shape[:2]
                    warped = cv2.warpAffine(sample_img, M, (w_r, h_r))
                    return warped
        except Exception as e:
            print(f"1차 실패: {e}")

    # 3-5. 두 번째 시도: 다양한 대비 향상 기법을 적용하여 SIFT 매칭 시도
    contrast_methods = ['clahe', 'hist_eq', 'gamma']

    for contrast_method in contrast_methods:
        try:
            enhanced_s = enhance_image_contrast(gray_s, contrast_method)
            enhanced_r = enhance_image_contrast(gray_r, contrast_method)

            kp_s, des_s = detector.detectAndCompute(enhanced_s, None)
            kp_r, des_r = detector.detectAndCompute(enhanced_r, None)

            if des_s is not None and des_r is not None:
                matches = bf.knnMatch(des_s, des_r, k=2)
                good = [m for m, n in matches if m.distance < 0.85 * n.distance]

                if len(good) >= 3:
                    src = np.float32([kp_s[m.queryIdx].pt for m in good]).reshape(-1, 2)
                    dst = np.float32([kp_r[m.trainIdx].pt for m in good]).reshape(-1, 2)
                    M, _ = cv2.estimateAffine2D(src, dst, method=cv2.RANSAC, ransacReprojThreshold=5.0)

                    if M is not None:
                        h_r, w_r = ref_img.shape[:2]
                        warped = cv2.warpAffine(sample_img, M, (w_r, h_r))
                        return warped

        except Exception as e:
            print(f"2차 실패 ({contrast_method}): {e}")

    # 3-6. 세 번째 시도: 이미지 크기 유사성 기반 단순 리사이즈 적용
    try:
        h_r, w_r = ref_img.shape[:2]
        h_s, w_s = sample_img.shape[:2]

        if abs(h_r - h_s) < 50 and abs(w_r - w_s) < 50:
            warped = cv2.resize(sample_img, (w_r, h_r))
            return warped
    except:
        pass

    # 3-7. 모든 정합 시도가 실패한 경우 None 반환
    return None

##### 메인 코드 #####
def run_enhanced_inspection():
  
    preds = []        # 최종 예측 결과 저장 리스트
    failed_files = [] # 정합 실패 파일 저장 리스트

    # 1-1. (gt_df)에서 각 이미지 순차 처리
    for filename in tqdm(gt_df['filename']):
        try:
            # 1-2. 샘플 이미지와 기준 이미지 불러오기
            sample_img, ref_img = load_single_sample_and_reference(filename, sample_folder, reference_folder)

            # 1-3. SIFT 기반 이미지 정합 수행
            warped = enhanced_sift_matching(sample_img, ref_img, filename)

            # 1-4. 모든 정합 시도 실패 시, 실패 파일 리스트에 기록하고 건너뛰기
            if warped is None:
                failed_files.append(filename)
                continue

            # 2-1. 정합된 이미지와 기준 이미지 간 차이 계산 (배경 영역 제외)
            mask_valid = np.any(warped != 0, axis=2)
            diff_full = cv2.absdiff(ref_img, warped)
            diff = np.zeros_like(diff_full)
            diff[mask_valid] = diff_full[mask_valid]

            # 2-2. 차이 이미지 전처리: 회색조 변환 → 이진화 → 잡음 제거
            gray_d = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
            _, thr = cv2.threshold(gray_d, 10, 255, cv2.THRESH_BINARY)
            thr = cv2.morphologyEx(thr, cv2.MORPH_OPEN, np.ones((4, 3), np.uint8))

            # 2-3. 윤곽선(변화 영역) 추출 후 면적 기준(≥70px) 필터링
            contours, _ = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = [c for c in contours if cv2.contourArea(c) >= 70]

            # 3-1. 차이 영역 미발견 시 RGB 채널별 추가 탐색 시도
            if not contours:
                # 3-2. 각 RGB 채널별로 개별 처리
                channels_to_try = [
                    ("Blue_channel", diff[:, :, 0]),
                    ("Green_channel", diff[:, :, 1]),
                    ("Red_channel", diff[:, :, 2])
                ]

                for channel_name, channel_img in channels_to_try:
                    _, thr = cv2.threshold(channel_img, 8, 255, cv2.THRESH_BINARY)
                    thr = cv2.morphologyEx(thr, cv2.MORPH_OPEN, np.ones((4, 3), np.uint8))

                    temp_contours, _ = cv2.findContours(thr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                    temp_contours = [c for c in temp_contours if cv2.contourArea(c) >= 60]

                    # 3-3. 윤곽선 발견 시 contours 업데이트 후 루프 종료
                    if temp_contours:
                        contours = temp_contours
                        break

            # 3-4. 추가 탐색 후에도 변화 영역 없으면 실패 처리
            if not contours:
                failed_files.append(filename)
                continue

            # 4-1. 이미지 중심에 가장 가까운 컨투어 선택
            h, w = gray_d.shape
            center_x, center_y = w / 2.0, h / 2.0

            def center_distance(cnt):
                M = cv2.moments(cnt)
                if M['m00'] == 0:
                    return float('inf')
                cx = M['m10'] / M['m00']
                cy = M['m01'] / M['m00']
                return (cx - center_x) ** 2 + (cy - center_y) ** 2

            c = min(contours, key=center_distance)

            # 5-1. 형태 기반 특징 추출 (외접 최소 사각형, 면적, 둘레, 원형도 등)
            rect = cv2.minAreaRect(c)
            (w_rec, h_rec) = rect[1]
            max_axis = max(w_rec, h_rec)
            min_axis = min(w_rec, h_rec)

            area = cv2.contourArea(c)
            perimeter = cv2.arcLength(c, True)
            ratio = 4 * np.pi * area / (perimeter ** 2 + 1e-6)

            hull = cv2.convexHull(c)
            hull_area = cv2.contourArea(hull)
            solidity = area / (hull_area + 1e-6)
            corner_count = len(c)

            rect_area = w_rec * h_rec
            rectangularity = area / (rect_area + 1e-6)

            # 5-2. 타원 근사 가능 시 이심률 계산
            if len(c) >= 5:
                ellipse = cv2.fitEllipse(c)
                major, minor = max(ellipse[1]) / 2, min(ellipse[1]) / 2
                eccentricity = np.sqrt(1 - (minor / (major + 1e-6)) ** 2)
            else:
                eccentricity = 1.0

            # 6-1. 밝기 기반 특징 추출 (불량 영역 마스크로 영역 밝기 평균 계산)
            mask = np.zeros_like(gray_d)
            cv2.drawContours(mask, [c], -1, 255, cv2.FILLED)

            mean_val = cv2.mean(cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY), mask=mask)[0]
            ref_gray = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY)
            ref_mean_val = cv2.mean(ref_gray, mask=mask)[0]

            # 7-1. 추출된 특징을 이용하여 불량 유형 최종 분류
            cls = classify_defect_with_reference_comparison(
                mean_val, ref_mean_val, eccentricity, max_axis, min_axis,
                solidity, ratio, ref_img, mask, c, corner_count, rectangularity
            )

            # 7-2. 결과를 최종 예측 리스트에 추가
            preds.append(cls)

        except Exception as e:
            # 8-1. 처리 중 예외 발생 시 실패 파일로 기록 후 다음 이미지로 진행
            print(f"[오류] {filename} - {e}")
            failed_files.append(filename)

    # 9-1. 모든 이미지 처리 완료 후 최종 예측 결과와 실패 리스트 반환
    return preds, failed_files


In [ ]:
# 1. 검사 실행 → 결함 분류 결과(preds)와 처리 실패 파일 목록(failed_files) 반환
preds, failed_files = run_enhanced_inspection()

# 2. 처리에 성공한 파일 목록을 생성해서 gt_df에 매칭하였음 / contour 추출 실패한 파일은 제외하고 평가됨
successful_files = [fname for fname in gt_df['filename'] if fname not in failed_files]
gt_df = gt_df[gt_df['filename'].isin(successful_files)]

# 3. 예측 결과 데이터프레임 생성
pred_df = pd.DataFrame({
    'filename': successful_files,
    'class':    preds
})

# 평가 (분류 정확도)

* 코드 수정은 하지말고 실행만 하세요.

In [ ]:
overall_acc, class_acc = evaluate_accuracy(gt_df, pred_df)

# 전체 Accuracy 출력
print(f"\n[전체 Accuracy]")
print(f"Overall Accuracy: {overall_acc * 100:.2f}%\n")

# 클래스별 Accuracy 출력
print("[클래스별 Accuracy]")
for cls, acc in sorted(class_acc.items()):
    print(f"Class '{cls}': {acc * 100:.2f}%")
